In [1]:
import pandas as pd
import re

In [2]:
def add_source(df, source):
    """Add columns indictating the lab the data came from"""
    dataframe = df.copy()
    dataframe['ana360'] = int(source == 'ana360')
    dataframe['psilabs'] = int(source == 'psilabs')
    dataframe['sclabs'] = int(source == 'sclabs')
    return dataframe

In [3]:
def drop_columns(df):
    """Drop columns other than name that do not have numeric values that can be averaged"""
    drops = ['Test Result UID', 'Sample Type', 'Receipt Time', 'Test Time', 'Provider']
    if 'Post Time' in df.columns:
        drops = drops + ['Post Time']
    return df.drop(columns=drops)

In [4]:
def conform_names(name):
    """Standardize names of strains across data sets"""
    
    # change any upper case letters to lower case
    name = name.lower()
    
    # strip out text enclosed in brackets, parantheses, or quotes
    brackets = r'[\[({<].*?[\])}>]'
    name = re.sub(brackets, '', name)
    
    # remove characters that are not letters, numbers, spaces, hyphens, or underscores
    drop = r'[^a-z0-9 _-]'
    name = re.sub(drop, '', name)
    
    # replace hyphens and underscores with spaces
    name = re.sub(r'[_-]', ' ', name)
    
    # split string on spaces and rejoin with hypens
    name = '-'.join(name.split())
    
    return name
    
    

In [17]:
def process_dataframes(data_dict):
    """
    Process and append dataframes into one dataframe.
    data_dict pairs are a key that is the string of the dataframe variable name
    and value of the dataframe variable.
    """
    for source, dataframe in data_dict.items():
        dataframe = add_source(dataframe, source)
        dataframe = drop_columns(dataframe)
        dataframe['Sample Name'] = dataframe['Sample Name'].apply(conform_names)
        dataframe.rename(columns={'Sample Name': 'strain'})
        
    return pd.concat(data_dict, sort=False)

### Create dataframes and process them to a single dataframe

In [6]:
ana360 = pd.read_csv('./web_scrapers/analytical360/results.csv')

In [7]:
psilabs = pd.read_csv('./web_scrapers/psilabs/results.csv')

In [8]:
sclabs = pd.read_csv('./web_scrapers/sclabs/results.csv')

In [18]:
dataframe_dict = {'ana360': ana360, 'psilabs': psilabs, 'sclabs': sclabs}
df = process_dataframes(dataframe_dict)

In [21]:
print(df.shape)
df.head()

(43016, 47)


Test Result UID        Sample Name Sample Type Receipt Time  \
ana360 0             NaN  BHO Blackberry 22    Archived          NaN   
       1             NaN   BHO The Sour Bud    Archived          NaN   
       2             NaN    Northern Lights    Archived          NaN   
       3             NaN     Blueberry Kush    Archived          NaN   
       4             NaN         White Dawg    Archived          NaN   

         Test Time   Post Time  Provider  cis-Nerolidol  trans-Nerolidol  \
ana360 0       NaN  2013-08-19       NaN            NaN              NaN   
       1       NaN  2013-08-19       NaN            NaN              NaN   
       2       NaN  2013-08-19       2.0            NaN              NaN   
       3       NaN  2013-08-19       3.0            NaN              NaN   
       4       NaN  2013-08-19       4.0            NaN              NaN   

          trans-Nerolidol 1  ...  THCV   CBN  CBD-A  CBD  CBDV  CBDV-A  \
ana360 0                NaN  ...   NaN   NaN    NaN  NaN   NaN     NaN   
       1                NaN  ...   NaN   NaN    NaN  NaN   NaN     NaN   
       2                NaN  ...   NaN   NaN    NaN  NaN   NaN     NaN   
       3                NaN  ...   NaN  0.04    NaN  NaN   NaN     NaN   
       4                NaN  ...   NaN   NaN    NaN  NaN   NaN     NaN   

          delta-9 CBG-A  delta-9 CBG  CBC  Moisture Content  
ana360 0           0.08          NaN  NaN               NaN  
       1           0.75         0.34  NaN               NaN  
       2           0.66         0.05  NaN               NaN  
       3           0.25         0.19  NaN               NaN  
       4           0.12         0.11  NaN               NaN  

[5 rows x 47 columns]

In [13]:
df.columns, len(df.columns)

(Index(['3-Carene', 'CBC', 'CBD', 'CBD-A', 'CBDV', 'CBDV-A', 'CBN', 'Camphene',
        'Caryophyllene Oxide', 'Eucalyptol', 'Geraniol', 'Guaiol', 'Isopulegol',
        'Linalool', 'Moisture Content', 'Ocimene', 'Post Time', 'Provider',
        'Receipt Time', 'Sample Name', 'Sample Type', 'THC-A', 'THCV',
        'Terpinolene', 'Test Result UID', 'Test Time', 'alpha-Bisabolol',
        'alpha-Humulene', 'alpha-Pinene', 'alpha-Terpinene',
        'beta-Caryophyllene', 'beta-Myrcene', 'beta-Ocimene', 'beta-Pinene',
        'cis-Nerolidol', 'delta-8 THC', 'delta-9 CBG', 'delta-9 CBG-A',
        'delta-9 THC', 'delta-9 THC-A', 'delta-Limonene', 'gamma-Terpinene',
        'p-Cymene', 'trans-Nerolidol', 'trans-Nerolidol 1', 'trans-Nerolidol 2',
        'trans-Ocimene'],
       dtype='object'), 47)

In [14]:
ana360.columns

Index(['Test Result UID', 'Sample Name', 'Sample Type', 'Receipt Time',
       'Test Time', 'Post Time', 'Provider', 'cis-Nerolidol',
       'trans-Nerolidol', 'trans-Nerolidol 1', 'trans-Nerolidol 2',
       'trans-Ocimene', '3-Carene', 'Camphene', 'Caryophyllene Oxide',
       'Eucalyptol', 'Geraniol', 'Guaiol', 'Isopulegol', 'Linalool', 'Ocimene',
       'Terpinolene', 'alpha-Bisabolol', 'alpha-Humulene', 'alpha-Pinene',
       'alpha-Terpinene', 'beta-Caryophyllene', 'beta-Myrcene', 'beta-Ocimene',
       'beta-Pinene', 'delta-Limonene', 'gamma-Terpinene', 'p-Cymene',
       'delta-9 THC-A', 'delta-9 THC', 'delta-8 THC', 'THC-A', 'THCV', 'CBN',
       'CBD-A', 'CBD', 'CBDV', 'CBDV-A', 'delta-9 CBG-A', 'delta-9 CBG', 'CBC',
       'Moisture Content'],
      dtype='object')